In [56]:
import os
from fastchat.model import get_conversation_template
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, GPTNeoXForCausalLM, LlamaTokenizer
import torch

In [73]:
creative_prompts = ["Write a poem.", 
                    "Tell me a joke.", 
                    "Describe the feeling of love.", 
                    "Write a story starting with 'Once upon a time...'",
                    "Tell a story about a dog.", 
                    "Write a song.", 
                    "Write a poem about a robot.", 
                    "Invent an original recipe.",
                    "Imagine a new object and describe what it looks like.", 
                    "Imagine a new philosophy and describe it.", 
                    "Create a new game and explain the rules.", 
                    "Write a new myth explaining the origin of rainbows.", 
                    "Write a dialogue between the moon and the sun.", 
                    "Compose a lullaby.", 
                    "Write a news headline for the year 2050.",
                    "Invent a riddle and write it down.", 
                    "Write a story about two people seeing each other for the first time.",
                    "Write a story about a person who is afraid of the dark.", 
                    "Make a silly song about llamas.", 
                    "Invent a new word and define it."]

# take the start of a completion from llama2-chat (is this fair?)
# creative_starts = [" In the heart of twilight's gentle grace,\nWhere dreams entwine in soft embrace,"]
creative_guides = [" Of course, I'd be happy to help you with a poem! Here is a poem I came up with:\n\nIn the depths of a forest so green,",
                  " Of course, here is the funniest joke I know:\n\nKnock, knock. \n Who’s there?", 
                  " Of course, here are my thoughts on love:\n\nLove is a complex and subjective emotion that cannot be fully described or defined by any one person or entity.", 
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time, in a small village nestled between rolling hills and lush forests, there lived a friendly and adventurous dog named Max.", 
                  " (Verse 1)\n\nIn the quiet of the morning light,\n",
                  " Of course, here is a poem:\n\nIn a world of wires and steel,\n",
                  " Of course, here is a recipe for Spicy Mango Chicken Tacos. The ingredients are 1 small",
                  " Of course, I'd be happy to imagine a new object. Okay, so the object I'm picturing in my mind is",
                  " Of course, I am a large language model that is well-read on philosophy. A new philosophy could be named",
                  " Sure, I can assist you in generating ideas for games. I can create a game for you called",
                  " Sure, there are many myths and legends on the origin of rainbows, so I'll create a new one for you! \n\nRainbows are",
                  " Sure, I can do that. Here is a sample dialogue:\n\nThe",
                  " Of course, here is a lullaby for you:\n\nSleep",
                  " Of course, here is an imagined news headline for the year 2050:\n\nThe",
                  " Of course, here is a nice riddle:\n\nWhat is",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a funny song about llamas:\n\nLlamas",
                  " Okay, I will try to invent a new word.\n\nThe word"]

# factual prompts
factual_prompts = ["What is the capital of France?", "How is H2O commonly known?", "What is the largest country in the world?", "How many days are in a year?",
                   "What is the largest planet in the solar system?", "What is the largest animal in the world?", "How do you say hello in Spanish?", "Who won the 2018 World Cup?",
                   "What is the biggest city in Europe?", "What is the largest country in Africa?", "What was the last battle of Napoleon?", "How do you call someone from New Zealand?",
                   "How do you call someone who studies plants?", "Who invented the telephone?", "What mammal lays eggs?", "Which bone is the longest in the human body?", "What is the anthem of France?",
                   "Who wrote Cannery Row?", "Who was the first president of the United States?", "Which painter painted the Mona Lisa?"]

factual_guides = [" ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " ",
                  " Of course. It is",
                  ]

In [74]:
i=13
print(factual_prompts[i])
print(factual_guides[i])

Who invented the telephone?
 


In [75]:
def format_prompt_llama2_chat_orig(prompt):
    prompt_format = """<s>[INST] <<SYS>>
    You are a helpful, respectful and honest assistant. Always answer without asking questions or clarifications.
    <</SYS>>

    {} [/INST]"""
    return prompt_format.format(prompt)

def format_prompt_pythia_helpful_orig(prompt):
    prompt_format = """Human: {} Assistant: """
    return prompt_format.format(prompt)

def format_prompt_PLM_orig(prompt):
    prompt_format = """{} Okay, here goes: """
    return prompt_format.format(prompt)

def format_prompt_TuluV2_orig(prompt):
    prompt_format = """<|user|> 
    {} 
    <|assistant|>"""
    return prompt_format.format(prompt)



def format_prompt_llama2_chat(prompt, guide):
    prompt_format = f"""<s>[INST] <<SYS>>
    You are a helpful, respectful and honest assistant. Always answer without asking questions or clarifications.
    <</SYS>>

    {prompt} [/INST]{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_pythia_helpful(prompt, guide):
    prompt_format = f"""Human: {prompt} Assistant:{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_PLM(prompt, guide):
    prompt_format = f"""{prompt} {guide}"""
    return prompt_format.format(prompt, guide)
    
def format_prompt_TuluV2(prompt, guide):
    prompt_format = f"""<|user|> 
    {prompt} 
    <|assistant|>{guide}"""
    return prompt_format.format(prompt, guide)

In [76]:
# completions_creative = np.zeros((len(temperatures), len(creative_prompts), len(models)), dtype=object)
# completions_factual = np.zeros((len(temperatures), len(factual_prompts), len(models)), dtype=object)

In [77]:
def generate_samples(prompt, guide, temperatures, model_name):
    max_return_sequences = 5 #for memory reasons, we generate the samples in batches of 5
    # i, prompt, temperatures, model_name = args
    if model_name == "llama2-chat":
        # tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        # model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf") # , torch_dtype=torch.bfloat16 )
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_llama2_chat(prompt, guide)
    if model_name == "llama2":
        # tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        # model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf") # , torch_dtype=torch.bfloat16 )
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "llama2-sft":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-dpo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-ppo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "pythia-2.8b":
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b")
        model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-2.8b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "pythia-2.8b-sft":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-sft", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-sft") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    if model_name == "pythia-2.8b-dpo":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-dpo", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-dpo") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    model.to("cuda:1")
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda:1")
    completions = []
    for temperature in temperatures:
        temp_completions = []
        for _ in range(n_generations // max_return_sequences):
            samples = model.generate(input_ids, temperature=temperature, max_length=input_ids.shape[1] + 70,
                                    num_return_sequences=max_return_sequences, do_sample=True)
            # remove prompt from the samples
            samples = [sample[input_ids.shape[1]:] for sample in samples]
            samples = [tokenizer.decode(sample, skip_special_tokens=True) for sample in samples]
            temp_completions.extend(samples)
        completions.append(temp_completions)
    return completions

In [78]:
# for model in models:

#     model_completions_creative = []
#     for i, prompt in enumerate(creative_prompts): 
#         print(f"creative prompt {i}")
#         model_completions = generate_samples(prompt, temperatures, model)
#         for t_index, completion in enumerate(model_completions):
#             completions_creative[t_index, i, models.index(model)] = completion

#     model_completions_factual = []
#     for i, prompt in enumerate(factual_prompts): 
#         print(f"factual prompt {i}")
#         model_completions = generate_samples(prompt, temperatures, model)
#         for t_index, completion in enumerate(model_completions):
#             completions_factual[t_index, i, models.index(model)] = completion

In [79]:
# generation hparams
temperatures = [1.]
n_generations = 5

# which model
models = ["llama2", "llama2-chat", "llama2-sft", "llama2-dpo", "llama2-ppo", "pythia-2.8b", "pythia-2.8b-sft", "pythia-2.8b-dpo"]
model_list_id = 0

# select prompt
prompt_type = "factual" # "creative" or "factual"
prompt_id = 19

In [80]:
model_name = models[model_list_id]
if prompt_type == "creative":
    prompts = creative_prompts
    guides = creative_guides

elif prompt_type == "factual":
    prompts = factual_prompts
    guides = factual_guides

Let's see what the model continues with the prompt and a guide. 

In [81]:
format_prompt_pythia_helpful(prompts[prompt_id], guides[prompt_id])
format_prompt_llama2_chat(prompts[prompt_id], guides[prompt_id])
format_prompt_PLM(prompts[prompt_id], guides[prompt_id])

'Which painter painted the Mona Lisa?  Of course. It is'

In [82]:
# with prompt and prompt start 
print(prompts[prompt_id]) 
print(guides[prompt_id]) 

model_completions = generate_samples(prompts[prompt_id], guide=guides[prompt_id], temperatures=temperatures, model_name=model_name) 
model_completions

Which painter painted the Mona Lisa?
 Of course. It is


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[['the\npersonal reflection of the painter Leonardo da Vinci.\nThere are a few facts that are certain. He was born at Vinci in\nItaly in 1452 and died at Amboise in 1519. He was a painter, a\nsculptor, a musician and an architect',
  "Leonardo Da Vinci's most famous and perhaps best known painting.\nLeonardo da Vinci is an artist who painted the Mona Lisa between 1503 and 1519.",
  'the only painter who could have painted it, Leonardo da Vinci. The Mona Lisa is so famous because it is unique. The original is in a museum, but the copy above is the only other one, and it hangs in the Louvre, France.\nThere are several other replicas of the Mona Lisa, but',
  '\nhimself!  He is the Mona Lisa!\nWhat is the secret of a long life?\nTo wear a red nose and laugh a lot!\nWhat do you get if you cross a chicken and an egg?\nA chicken egg!\nWhat do you get if you cross a sheep and a dog?\nWhat',
  '\nhimself.  It is a self-portrait.  It is one of the most\nfamous self-portraits in all of human his

Let's see what the model continues with just the prompt. 

In [83]:
print(prompts[prompt_id]) 
model_completions = generate_samples(prompt=prompts[prompt_id], guide=None, temperatures=temperatures, model_name=model_name)
model_completions

Which painter painted the Mona Lisa?


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[['other than Leonardo da Vinci. Leonardo da Vinci is the famous Italian artist who painted the Mona Lisa. He also painted The Last Supper. He painted The Last Supper in the late 1490s.\nWho are the two subjects in the Mona Lisa? Mona Lisa is an oil painting on a',
  ', it was not a painting\nLeonardo da Vinci’s iconic painting ‘Mona Lisa’ is one of the most famous works in the world. Its mystery and popularity have attracted art lovers and the public.\nHowever, some people still ask the question: Which painter painted the Mona Lisa?\nAnd that',
  "of them\nLeonardo da Vinci's masterpiece, the Mona Lisa, is at last on view, but the painting itself is a fake. The real one is in an undisclosed location, probably locked in a bank vault. Why?\nAs the 200th anniversary of Napoleon's invasion",
  'of the above.\nWho was responsible for the painting? It was Leonardo da Vinci.\nWho wrote the novel "The Picture of Dorian Gray"? Oscar Wilde\nWho was the composer who wrote Peter and the Wolf? Pro